## Team Member Names: Madeline Witters {-}

## Project Title: Predicting Customer Churn and Identifying Attributes of At-Risk Customers {-}

**Exploratory Data Analysis**

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.linear_model import LogisticRegressionCV
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import PrecisionRecallDisplay

In [ ]:
data = pd.read_csv("Bank Customer Churn Prediction.csv")
data = data.drop("customer_id", axis=1)
data.head()

In [ ]:
data.dtypes

In [ ]:
#EDA to do list: 
# drop customer_id column DONE
# check for missing data DONE
# create boxplots for categorical variables DONE
# create density plots/histograms for quantitative vars DONE
#One Hot encoding: gender, country, Balance DONE

#info on why it's best to not do too much one-hot encoding for trees: 
#https://towardsdatascience.com/one-hot-encoding-is-making-your-tree-based-ensembles-worse-heres-why-d64b282b5769

#11/17 TO DO LIST: 

#outlier removal (do before standardization): 
#https://medium.com/geekculture/essential-guide-to-handle-outliers-for-your-logistic-regression-model-63c97690a84d

#check for multicollinearity: corr plot? DONE

#create mosaic plots for categorical vars created bar plots -- need to create mosaic plots

#create additional boxplots for numerical vars DONE

#split into train/test DONE

#standardize data DONE

#Lasso variable selection DONE (conduct research on what vars to include: perhaps simply include all for now ?)

#Create Logistic regression model

#Create RF model
#   -identify most important vars in RF

In [ ]:
null_check = data.isnull().any() #no missing data in the dataframe
null_check

In [ ]:
#data.min()

In [ ]:
#data.max()

In [ ]:
plt.figure(figsize=(20,20))
#plt.title("Boxplots of Numeric Dependent Variables") try to add in tile later ?

plt.subplot(3,2,1)
sns.boxplot(x='churn', y='credit_score', data=data)

plt.subplot(3,2,2)
sns.boxplot(x='churn', y='age', data=data)

plt.subplot(3,2,3)
sns.boxplot(x='churn', y='tenure', data=data)

plt.subplot(3,2,4)
sns.boxplot(x='churn', y='balance', data=data)

plt.subplot(3,2,5)
sns.boxplot(x='churn', y='products_number', data=data)

plt.subplot(3,2,6)
sns.boxplot(x='churn', y='estimated_salary', data=data)

In [ ]:
sns.histplot(data=data, x="balance", kde=True)

In [ ]:
sns.histplot(data=data, x="age", kde=True)

In [ ]:
credit_score_zs = stats.zscore(data['credit_score'])

In [ ]:
age_z = stats.zscore(data['age'])

In [ ]:
ten_z = stats.zscore(data['tenure'])
print(ten_z)

In [ ]:
bal_z = stats.zscore(data['balance'])
print(bal_z)

In [ ]:
prod_z = stats.zscore(data['products_number'])
print(prod_z)

In [ ]:
sal_z = stats.zscore(data['estimated_salary'])
print(sal_z)

In [ ]:
threshold = 3
outlier = [] #write in report about how I conducted outlier analysis, and why I decided not to exclude any points
for z in age_z: #further address how this could be modeled in the future (segment customers by age, products number) + build more models
    if z > threshold: #this could be addressed in conclusion or EDA section
        outlier.append(z)
#print('outlier in dataset is', outlier)
#print(len(outlier))

In [ ]:
data['churn'].value_counts()

In [ ]:
plt.figure(figsize=(20,20))
#plt.title("Boxplots of Numeric Dependent Variables") try to add in tile later ?

plt.subplot(3,2,1)
sns.countplot(data=data, x="country", hue="churn")

plt.subplot(3,2,2)
sns.countplot(data=data, x="gender", hue="churn")

plt.subplot(3,2,3)
sns.countplot(data=data, x="credit_card", hue="churn")

plt.subplot(3,2,4)
sns.countplot(data=data, x="active_member", hue="churn")

In [ ]:
crosstable = pd.crosstab(data['churn'], data['gender'])
crosstable

In [ ]:
mosaic(data, ['country', 'churn'], title="Country by Churn")
mosaic(data, ['gender', 'churn'], title="Gender by Churn")
labelizer = lambda k: {('0','0'): 'no credit_card', ('0','1'): 'no credit_card', ('1','0'): 'credit_card',('1','1'): 'credit_card'}[k]
mosaic(data, ['credit_card', 'churn'], labelizer =labelizer, title="Credit Card by Churn")
labels = lambda k: {('0','0'): 'non-active member', ('0','1'): 'non-active member', ('1','0'): 'active_member',('1','1'): 'active_member'}[k]
mosaic(data, ['active_member', 'churn'], labelizer=labels, title="Active Member by Churn")

In [ ]:
g_one_hot = pd.get_dummies(data['gender'])
g_one_hot.head()

In [ ]:
data = data.drop('gender',axis = 1)
# Join the encoded df
data = data.join(g_one_hot)
data.head()

In [ ]:
c_one_hot = pd.get_dummies(data['country'])
#c_one_hot.head()

In [ ]:
data2 = data.drop('country',axis = 1)
data2 = data2.join(c_one_hot)
#data2.head()

In [ ]:
data2['zero_balance'] = np.where(data2['balance'] == 0.0, 1, 0) #leave this for later on! 

In [ ]:
#data2.head()

In [ ]:
data2 = data2.drop('balance',axis = 1)

In [ ]:
first_column = data2.pop('churn')
data2.insert(0, 'churn', first_column)

In [ ]:
data2.head()

In [ ]:
data2.dtypes

In [ ]:
numeric_vars = data2[['churn', 'credit_score', 'age', 'tenure','products_number', 'estimated_salary']]

In [ ]:
corr_matrix = numeric_vars.corr().round(2)
print(corr_matrix)

In [ ]:
plt.figure(figsize = (14,7))
sns.heatmap(corr_matrix, annot=True, cmap='Blues')
plt.title(label="Correlation Matrix for Numeric Variables")
plt.show()

**Variable Selection**

In [ ]:
data2.head()

In [ ]:
features = data2.columns[1:14]
target = data2.columns[0]
X = data2[features].values
y = data2[target].values

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [ ]:
np.unique(y_train, return_counts=True)

In [ ]:
np.unique(y_test, return_counts=True)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

In [ ]:
lasso_glm = LogisticRegressionCV(Cs = [0.001, 0.005, 0.0075, 0.01, .05, .075, .1, .5, .75, 1], cv=10, penalty='l1', solver="liblinear", random_state=2).fit(X_train_scaled, y_train)

In [ ]:
lasso_glm.C_

In [ ]:
cs = lasso_glm.Cs_
print(cs)

In [ ]:
scs = lasso_glm.scores_[1]
scs

In [ ]:
scores = np.mean(scs, axis=0)
scores

In [ ]:
plt.plot(cs, scores, "-o") #Inverse of regularization strength; must be a positive float. Smaller values specify stronger regularization.
plt.title("Average Accuracy Rate for Each Fold")
plt.xlabel("C tuning parameter")
plt.ylabel('Accuracy/Score for 0.5 Threshold')

In [ ]:
lasso_glm.coef_

In [ ]:
pd.Series(lasso_glm.coef_[0], features).sort_values(ascending = True).plot(kind = "bar")
plt.title("Magnitude of Coefficients Determined by Logistic Lasso")

**Modeling**

*Logistic Regression*

In [ ]:
#13 features: remove : 9, 10, 12 (one-indexed, correct for zero based)

In [ ]:
print(X_train)
X_train.shape

In [ ]:
X_train_mod = np.delete(X_train, [8,9,11], axis=1)

In [ ]:
X_train_mod.shape

In [ ]:
print(X_train_mod)

In [ ]:
X_test_mod = np.delete(X_test, [8,9,11], axis=1)

In [ ]:
X_test_mod.shape #dropped irrelevant vars, but did not scale

In [ ]:
log = LogisticRegression(random_state=0).fit(X_train_mod, y_train)

In [ ]:
log.coef_

In [ ]:
log.score(X_train_mod, y_train)

In [ ]:
log.score(X_test_mod, y_test) #accuracy for test data

In [ ]:
#Citation: https://stackoverflow.com/questions/28716241/controlling-the-threshold-in-logistic-regression-in-scikit-learn

In [ ]:
ax = plt.gca()
plt.plot([0, 1], [0, 1], linestyle="--", label='No Skill')
log_disp = RocCurveDisplay.from_estimator(log, X_test_mod, y_test, ax=ax, alpha=0.8)
plt.title("ROC Curve for Test Data")
plt.show()

In [ ]:
probs_y=log.predict_proba(X_test_mod) 

display = PrecisionRecallDisplay.from_predictions(y_test, probs_y[:,1], name="LogisticRegression")
_ = display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
pred_proba = log.predict_proba(X_test_mod)[:,1]
#print(pred_proba)
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95]
for i in threshold_list:
    print ('\n******** For i = {} ******'.format(i))
    y_test_pred = np.where(pred_proba > i, 1, 0)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print('Accuracy/Score is {}'.format(test_accuracy))
    print(confusion_matrix(y_test, y_test_pred))
    print(classification_report(y_test, y_test_pred, zero_division=1))

In [ ]:
#pred_y=log.predict(X_test_mod) 

probs_y=log.predict_proba(X_test_mod) 
  # probs_y is a 2-D array of probability of being labeled as 0 (first 
  #column of 
  #array) vs 1 (2nd column in array)

precision, recall, thresholds = precision_recall_curve(y_test, probs_y[:, 
1]) 
   #retrieve probability of being 1(in second column of probs_y)
pr_auc = auc(recall, precision)

plt.title("Precision-Recall vs Threshold Chart (Logistic Model)")
plt.plot(thresholds, precision[: -1], label="Precision")
plt.plot(thresholds, recall[: -1], label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

In [ ]:
#based off plot above, a threshold around 0.3 may produce the best prediction performance

*Random Forest*

In [ ]:
# error_rates = [] #varying number of trees and number of features did not appear to significantly impact RF performance

# num_trees = range(50,120)
# for i in num_trees:
#     rf = RandomForestClassifier(n_estimators = i, random_state=0)
#     rf = rf.fit(X_train_mod, y_train)
#     rf_pred = rf.predict(X_test_mod)
#     rf_accuracy = accuracy_score(y_test, rf_pred)
#     error_rates.append(rf_accuracy)
    
# print(error_rates)

In [ ]:
# plt.plot(num_trees, error_rates, '-o', label="RF Error Rates") 
# plt.title("Accuracy Against Number of Trees")
# plt.xlabel("RF Number of Trees")
# plt.ylabel('Accuracy')
# plt.grid()
# plt.legend(loc="upper right")

In [ ]:
rf = RandomForestClassifier(random_state=0).fit(X_train_mod, y_train)

In [ ]:
rf_preds = rf.predict(X_test_mod)

In [ ]:
rf_accuracy = accuracy_score(y_test, rf_preds)
print('Accuracy/Score is {}'.format(rf_accuracy))
print(confusion_matrix(y_test, rf_preds))
print(classification_report(y_test, rf_preds, zero_division=1))

In [ ]:
ax = plt.gca()
plt.plot([0, 1], [0, 1], linestyle="--", label='No Skill')
log_disp.plot(ax=ax, alpha=0.8)
rfc_disp = RocCurveDisplay.from_estimator(rf, X_test_mod, y_test, ax=ax, alpha=0.8)
plt.title("ROC Curve Logistic Regression vs RF")
plt.show()

In [ ]:
display = PrecisionRecallDisplay.from_estimator(
    rf, X_test_mod, y_test, name="RandomForest"
)
_ = display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
rf.feature_importances_

In [ ]:
#data2.columns.values.tolist()

In [ ]:
feature_names = [
 'credit_score',
 'age',
 'tenure',
 'products_number',
 'credit_card',
 'active_member',
 'estimated_salary',
 'Female',
 'Germany',
 'zero_balance']

feature_names = np.array(feature_names)
print(feature_names)

In [ ]:
importances = rf.feature_importances_
print(importances)
important_names = feature_names[importances > np.mean(importances)]
print(important_names)

In [ ]:
#https://towardsdatascience.com/feature-selection-using-random-forest-26d7b747597f#:~:text=The%20more%20a%20feature%20decreases,final%20importance%20of%20the%20variable.